In [24]:
import re
import nltk
import torch
import matplotlib.pyplot as plt
import statsmodels as sm
import numpy as np
import scipy
import sklearn
import pandas as pd

# Use a pipeline as a high-level helper
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [27]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [28]:
json_path="/Users/merrheimmaissane/Documents/TSP + IPP 3A/IPP/Data Stream Processing/Projet Kafka/infostream/eco-news/eco_articles.json"

# Load the data 
with open(json_path, 'r') as file:
    json_data = file.read()

# Convert the data into a DataFrame for better manipulation
data_eco = pd.read_json(json_data)
df_eco = pd.DataFrame(data_eco)

df_eco.head()

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': None, 'name': 'Yahoo Entertainment'}",Cecile Gutscher and Alex Nicholson,"Asia Stocks to Gain, Treasuries Rally on US Jo...",(Bloomberg) -- Stock markets ticked higher as ...,https://finance.yahoo.com/news/asia-stocks-gai...,https://media.zenfs.com/en/bloomberg_markets_8...,2023-12-06T09:52:37Z,(Bloomberg) -- Stock markets ticked higher as ...
1,"{'id': None, 'name': 'CNBC'}",Elliot Smith,'You can't just make your own rules': Tesla fa...,Tesla faces a growing revolt in Scandinavia af...,https://www.cnbc.com/2023/12/06/you-cant-just-...,https://image.cnbcfm.com/api/v1/image/10733510...,2023-12-06T09:31:55Z,Tesla faces a growing revolt in Scandinavia af...
2,"{'id': None, 'name': 'BBC News'}",None,Airline adverts banned over 'greenwashing' cla...,"The ASA said Air France, Lufthansa and Etihad ...",https://www.bbc.com/news/business-67625200,https://ichef.bbci.co.uk/news/1024/branded_new...,2023-12-06T08:41:25Z,"By Sam GruetBusiness reporter, BBC News\r\nAdv..."
3,"{'id': None, 'name': 'CNBC'}",Sam Meredith,A ‘thirsty’ generative AI boom poses a growing...,A global rush for the next wave of generative ...,https://www.cnbc.com/2023/12/06/water-why-a-th...,https://image.cnbcfm.com/api/v1/image/10734232...,2023-12-06T06:40:59Z,"Dubai, UNITED ARAB EMIRATES A global rush for ..."
4,"{'id': None, 'name': 'KABC-TV'}",None,California EDD drops Bank of America for Money...,Big news for those collecting unemployment pay...,https://abc7.com/california-edd-ca-unemploymen...,https://cdn.abcotvs.com/dip/images/10855336_bo...,2023-12-06T05:46:57Z,SAN FRANCISCO -- Big news for those collecting...


In [29]:
# Create a list of the article titles 
top_headlines = []
for i in range(len(df_eco)) : 
    top_headlines.append(df_eco["title"][i])

# Tokenize the article titles and run them through the model 
inputs = tokenizer(top_headlines, padding = True, truncation = True, return_tensors='pt')
outputs = model(**inputs)

# Compute the predictions of our model 
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

In [30]:
positive = predictions[:, 0].tolist()
negative = predictions[:, 1].tolist()
neutral = predictions[:, 2].tolist()


table = {'Headline':top_headlines,
         "Positive":positive,
         "Negative":negative, 
         "Neutral":neutral}
      
df_eco_news = pd.DataFrame(table, columns = ["Headline", "Positive", "Negative", "Neutral"])
df_eco_news

,Headline,Positive,Negative,Neutral
0,"Asia Stocks to Gain, Treasuries Rally on US Jo...",0.599681,0.239932,0.160386
1,'You can't just make your own rules': Tesla fa...,0.025079,0.797342,0.177578
2,Airline adverts banned over 'greenwashing' cla...,0.018098,0.583353,0.398549
3,A ‘thirsty’ generative AI boom poses a growing...,0.075185,0.406727,0.518089
4,California EDD drops Bank of America for Money...,0.055666,0.027179,0.917155
5,Swanky San Francisco house has lost $10 millio...,0.007180,0.957830,0.034990
6,Apple moves towards India-made iPhone batterie...,0.339026,0.017455,0.643518
7,Nvidia to develop new chips that comply with U...,0.198582,0.014194,0.787225
8,Elon Musk is raising money for his AI company ...,0.104643,0.009263,0.886094
9,SF business owners fed up with Valencia bike l...,0.034011,0.078049,0.887940
